# Sample Submission

The sample submission Python API is served through a single function: `parse_and_insert_df`.

`parse_and_insert_df` is ACID-compliant and does not allow for duplicate inserts.

In [ ]:
!lndb init --storage "testsample" --schema "bionty,wetlab"

In [ ]:
import pandas as pd
import numpy as np
import lamindb as ln
from lnschema_wetlab.dev import parse_and_insert_df

Let's instantiate a dataframe with test sample data.

In [ ]:
biosample = pd.DataFrame(
    {
        "Name": [
            "hc_dexameth",
            "cv_hydrocort_1",
            "cv_hydrocort_2",
            "hc_predniso",
            "hc_triamcino",
        ],
        "Species": ["human", "human", "human", "human", "human"],
        "Cell Type": ["CD8+T", "CD8+T", "CD8+T", "CD8+T", "CD8+T"],
        "Experiment": ["001", "002", "003", "004", "004"],
        "Donor": ["021", "027", "010", "049", "002"],
        "Disease": ["U07.1", "U07.1", "U07.1", "I51.9", "K83.9"],
        "Experiment": ["001", "002", "003", "004", np.nan],
        "Perturbation": ["A01AC02", "A01AC03", "A01AC03", "A01AC54", "A01AC01"],
        "Custom 1": ["healthy", "acute", "acute", "convalescent", "recovered"],
        "Custom 2": ["control", "covid-19", "covid-19", "control", "control"],
        "Custom 3": [12.11, np.nan, 0.87, np.nan, 11.91],
    }
)

techsample = pd.DataFrame(
    {
        "Name": ["TS001", "TS002", "TS003", "TS004", "TS005"],
        "Batch": [1, 1, 1, 2, 2],
        "File Type": ["fastq", "fastq", "fastq", "fastq", "fastq"],
        "Filepath R1": [
            "SRX1603629_T1_1.fastq.gz",
            "SRX1603629_T1_2.fastq.gz",
            "SRX1603629_T1_3.fastq.gz",
            "SRX1603629_T1_4.fastq.gz",
            "SRX1603629_T1_5.fastq.gz",
        ],
        "Filepath R2": [
            "SRX1603629_T2_1.fastq.gz",
            "SRX1603629_T2_2.fastq.gz",
            "SRX1603629_T2_3.fastq.gz",
            "SRX1603629_T2_4.fastq.gz",
            "SRX1603629_T2_5.fastq.gz",
        ],
        "Custom 1": ["13.42%", "2.43%", "4.57%", np.nan, "9.36%"],
        "Custom 2": [
            "Gene Expression",
            "Gene Expression",
            "Gene Expression",
            "Gene Expression",
            "Gene Expression",
        ],
    }
)

Let's now use `parse_and_insert_df` to process the dataframe and insert the relevant entries in their respective tables.

`parse_and_insert_df` takes two arguments as inputs:
* A `pandas.DataFrame` with the sample data.
* A string with the name of the target LaminDB table (e.g. "biosample", "techsample").

`parse_and_insert_df` currently makes three matching assumptions:
* **Table matching (input string to primary (sample) LaminDB table)**: perfect, case insensitive matching.
    * Custom schemas take preference over core schemas (core, wetlab, bionty, bfx). E.g `customer.Biosample` takes preference over `wetlab.Biosample`.
* **Column matching (input DataFrame columns to secondary LaminDB tables)**: perfect, case insensitive matching.
    * Insertion assumption: insertion column on the target table is the first one to contain the "name" substring.
* **Column matching (input DataFrame columns to primary (sample) LaminDB table columns)**: perfect, case insensitive matching.

In the future, we will be able to relax some of these assumptions by building an interactive sample submission UI, where the user will be able to customize mappings.

In [ ]:
res_biosample = parse_and_insert_df(biosample, "biosample")
res_biosample

In [ ]:
res_techsample = parse_and_insert_df(techsample, "techsample")
res_techsample

`parse_and_insert_df` returns a dictionnary with two items: column mappings and table entries.

**Column mappings** is a dictionnary with all mappings between the DataFrame column and their respective LaminDB tables and table columns.
* Key (str): dataframe column name
* Value (tuple): (table name, table column)

**Table entries** is a dictionnary with all added entries.
* Key (str): table name
* Value (list): list with all added LaminDB records

We can now check that the entries have been accurately inserted in the database.

In [ ]:
ln.view()